# Canonical Quantization of the Free Scalar Field

This notebook contains the programmatic verification for the **Canonical Quantization of the Free Scalar Field** entry from the THEORIA dataset.

**Entry ID:** scalar_field_quantization  
**Required Library:** sympy 1.13.1

## Description
Starting from the Klein-Gordon field and assuming canonical commutation relations, the mode operator algebra is derived and the Fock space of particle states is constructed. The ladder algebra of the number operator shows that mode operators raise or lower particle number, justifying the names creation and annihilation operators. The normal-ordered Hamiltonian provides the particle interpretation of quantum field theory, where each mode contributes energy proportional to its occupation number.

## Installation
First, let's install the required library:

In [ ]:
# Install required library with exact version
!pip install sympy==1.13.1

## Programmatic Verification

The following code verifies the derivation mathematically:

In [ ]:
import sympy as sp

# =====================================================
# Programmatic verification: Scalar Field Quantization
#
# Sections:
#  1. Setup: Define symbols
#  2. Step 3: Verify dispersion relation
#  3. Steps 16-17: Verify Hamiltonian reordering
#  4. Step 18: Verify ladder algebra
#  5. Step 19: Verify ladder action on eigenstates
#  6. Step 22: Verify normal ordering and particle energies
# =====================================================

# ---------------------------
# Section 1: Setup
# ---------------------------
kx, ky, kz = sp.symbols('k_x k_y k_z', real=True)
m, hbar = sp.symbols('m hbar', positive=True, real=True)
omega = sp.symbols('omega', positive=True, real=True)
n = sp.Symbol('n', integer=True, nonnegative=True)

# ---------------------------
# Section 2: Step 3 - Dispersion relation
# ---------------------------

# omega_k = sqrt(|k|^2 + m^2)
k_squared = kx**2 + ky**2 + kz**2
omega_k = sp.sqrt(k_squared + m**2)

# Verify dispersion relation satisfies omega^2 = k^2 + m^2
dispersion_check = omega_k**2 - k_squared - m**2
assert sp.simplify(dispersion_check) == 0, 'Step 3: Dispersion relation omega^2 = k^2 + m^2'

# For massless case (m=0): omega = |k|
omega_massless = omega_k.subs(m, 0)
assert sp.simplify(omega_massless - sp.sqrt(k_squared)) == 0, 'Step 3: Massless dispersion omega = |k|'

# ---------------------------
# Section 3: Steps 16-17 - Hamiltonian reordering
# ---------------------------

# Step 16: H = integral omega_k/2 * (a*a^dag + a^dag*a)
# Step 17: Using [a, a^dag] = 1 (discrete normalization for single mode)
# a*a^dag = a^dag*a + 1
# So (a*a^dag + a^dag*a) = 2*a^dag*a + 1
# H = integral omega_k * (a^dag*a + 1/2)

# Verify algebraically (using symbolic commutator)
a_dag_a = sp.Symbol('N', commutative=False)  # Number operator N = a^dag * a
a_a_dag = a_dag_a + 1  # From [a, a^dag] = 1

H_step16 = omega * sp.Rational(1, 2) * (a_a_dag + a_dag_a)
H_step16_expanded = omega * sp.Rational(1, 2) * (a_dag_a + 1 + a_dag_a)
H_step16_simplified = omega * sp.Rational(1, 2) * (2*a_dag_a + 1)
H_with_zpe = omega * (a_dag_a + sp.Rational(1, 2))

assert sp.simplify(H_step16_simplified - H_with_zpe) == 0, 'Step 17: H = omega*(N + 1/2)'

# ---------------------------
# Section 4: Step 18 - Ladder algebra
# ---------------------------

# Verify [N, a^dag] = a^dag and [N, a] = -a using non-commutative algebra
# Using N = a^dag * a and [a, a^dag] = 1

a, a_dag = sp.symbols('a a_dag', commutative=False)

# Define commutator function
def comm(X, Y):
    return sp.expand(X*Y - Y*X)

# Given: [a, a_dag] = 1, so a*a_dag = a_dag*a + 1
# We substitute a*a_dag -> a_dag*a + 1 to simplify

# Compute [N, a_dag] = [a_dag*a, a_dag] = a_dag*a*a_dag - a_dag*a_dag*a
N = a_dag * a
comm_N_adag = comm(N, a_dag)
# = a_dag*a*a_dag - a_dag*a_dag*a
# = a_dag*(a_dag*a + 1) - a_dag*a_dag*a  (using a*a_dag = a_dag*a + 1)
# = a_dag*a_dag*a + a_dag - a_dag*a_dag*a = a_dag

# Verify by substitution: replace a*a_dag with a_dag*a + 1
comm_N_adag_sub = comm_N_adag.subs(a*a_dag, a_dag*a + 1)
assert sp.expand(comm_N_adag_sub - a_dag) == 0, 'Step 18: [N, a_dag] = a_dag'

# Compute [N, a] = [a_dag*a, a] = a_dag*a*a - a*a_dag*a
comm_N_a = comm(N, a)
# = a_dag*a*a - a*a_dag*a
# = a_dag*a*a - (a_dag*a + 1)*a  (using a*a_dag = a_dag*a + 1)
# = a_dag*a*a - a_dag*a*a - a = -a

comm_N_a_sub = comm_N_a.subs(a*a_dag, a_dag*a + 1)
assert sp.expand(comm_N_a_sub + a) == 0, 'Step 18: [N, a] = -a'

# ---------------------------
# Section 5: Step 19 - Ladder action on eigenstates
# ---------------------------

# Verify the algebraic identity: N*a_dag = a_dag*N + [N, a_dag] = a_dag*N + a_dag
# If N|n> = n|n>, then N*a_dag|n> = (a_dag*N + a_dag)|n> = a_dag*n|n> + a_dag|n> = (n+1)*a_dag|n>

# Verify N*a_dag = a_dag*N + a_dag (after substitution)
lhs = N * a_dag
rhs = a_dag * N + a_dag
diff = sp.expand(lhs - rhs).subs(a*a_dag, a_dag*a + 1)
assert sp.expand(diff) == 0, 'Step 19: N*a_dag = a_dag*N + a_dag'

# Verify N*a = a*N + [N, a] = a*N - a
lhs_a = N * a
rhs_a = a * N - a
diff_a = sp.expand(lhs_a - rhs_a).subs(a*a_dag, a_dag*a + 1)
assert sp.expand(diff_a) == 0, 'Step 19: N*a = a*N - a'

# ---------------------------
# Section 6: Step 22 - Normal ordering and particle energies
# ---------------------------

# Normal ordering removes zero-point energy
H_normal_ordered = omega * a_dag_a

# The difference is the zero-point energy omega/2 per mode
zpe_per_mode = H_with_zpe - H_normal_ordered
assert sp.simplify(zpe_per_mode - omega * sp.Rational(1, 2)) == 0, 'Step 22: Zero-point energy is omega/2'

# Particle interpretation: H = sum_k omega_k * N_k
# Energy of state with n_k particles in mode k is sum_k n_k * omega_k

# Verify for specific configurations
n1, n2, n3 = sp.symbols('n1 n2 n3', integer=True, nonnegative=True)
omega1, omega2, omega3 = sp.symbols('omega1 omega2 omega3', positive=True)

total_energy = n1*omega1 + n2*omega2 + n3*omega3

# Vacuum: n1=n2=n3=0, E=0
E_vacuum = total_energy.subs([(n1, 0), (n2, 0), (n3, 0)])
assert E_vacuum == 0, 'Step 22: Vacuum energy is 0 (normal ordered)'

# Single particle in mode 1: n1=1, n2=0, n3=0, E=omega1
E_single = total_energy.subs([(n1, 1), (n2, 0), (n3, 0)])
assert E_single == omega1, 'Step 22: Single particle energy is omega_k'

# Two particles: n1=1, n2=1, n3=0, E=omega1+omega2
E_two = total_energy.subs([(n1, 1), (n2, 1), (n3, 0)])
assert E_two == omega1 + omega2, 'Step 22: Two particle energy is sum'

print('Scalar field quantization verification passed')


## Source

📖 **View this entry:** [theoria-dataset.org/entries.html?entry=scalar_field_quantization.json](https://theoria-dataset.org/entries.html?entry=scalar_field_quantization.json)

This verification code is part of the [THEORIA dataset](https://github.com/theoria-dataset/theoria-dataset), a curated collection of theoretical physics derivations with programmatic verification.

**License:** CC-BY 4.0